## Esse notebook apresenta o algoritmo que prevê o time que marcará o primeiro gol

Utilizando o algoritmo Random Forest Classifier, foi desenvolvido um modelo que prevê qual time marcará o primeiro gol em uma partida.

**Tutorial de como rodar esse notebook:**

1 - Primeiro, é necessário rodar o notebook 'notebooks/pre-processamento/pre-processamento.ipynb', para que as tabelas utilizadas neste arquivo sejam geradas corretamente.

2 - As bibliotecas necessárias já estão sendo importadas no código, mas se, por algum motivo, não funcionar, instale as bibliotecas no seu computador através do comando **pip install (nome da biblioteca)**

3 - Ter o Python instalado

Importação das bibliotecas

In [10]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
import matplotlib as plt

Carrega a tabela

In [11]:
# Carregar o arquivo CSV
df_teams = pd.read_csv(('../Dados/dfTeams.csv'))

Seleciona as colunas mais relevantes

Pensando que o objetivo do modelo é acertar qual time marcará o primeiro gol, é interessante selecionar colunas que representam o quão rápido os times costumam marcar, mas além disso, é importante expandir as variáveis para entender quais times costumam ganhar ou perder mais.

Vários testes foram realizados com adição e remoção de features, segue abaixo as selecionadas:

In [12]:
# Criando a variável-alvo: 1 se o time da casa marcou mais gols nos primeiros 10 minutos, 0 se foi o time visitante.
df_teams['first_goal_home'] = df_teams.apply(
    lambda row: 1 if row['goals_scored_min_0_to_10'] > row['goals_conceded_min_0_to_10'] else 0, axis=1
)

# Selecionando colunas relevantes para a modelagem
colunas_relevantes = [
    'goals_scored_min_0_to_10', 'goals_conceded_min_0_to_10',
    'goals_scored_min_11_to_20', 'goals_conceded_min_11_to_20', 'wins_home', 'wins_away',
    'loss_percentage_home', 'loss_percentage_away', 'over35_percentage'
]

# Criando um DataFrame com as colunas selecionadas e a variável-alvo
df_model = df_teams[colunas_relevantes + ['first_goal_home']]

# Removendo valores nulos
df_model = df_model.dropna()


Separa os dados entre treinamento e teste

In [13]:
# Separar os dados em treino e teste
X = df_model.drop('first_goal_home', axis=1)
y = df_model['first_goal_home']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(n_estimators=50, random_state=42)

# Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
y_pred = model.predict(X_test)


Correção das variáveis

In [14]:
# Usar validação cruzada com 5 divisões (folds)
scores = cross_val_score(model, X, y, cv=5)

# Exibir as métricas de cada divisão
print(f'Scores de validação cruzada: {scores}')
print(f'Média dos scores: {scores.mean()}')


Scores de validação cruzada: [0.75 1.   0.75 1.   0.75]
Média dos scores: 0.85


In [15]:
# Treinar o modelo Ridge com regularização
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)


Ridge()

In [16]:
# Calcular a matriz de correlação
corr_matrix = df_model.corr()

# Exibir a matriz de correlação
print(corr_matrix)


                             goals_scored_min_0_to_10  \
goals_scored_min_0_to_10                     1.000000   
goals_conceded_min_0_to_10                  -0.271039   
goals_scored_min_11_to_20                   -0.096884   
goals_conceded_min_11_to_20                 -0.089968   
wins_home                                    0.475858   
wins_away                                   -0.094042   
loss_percentage_home                        -0.201746   
loss_percentage_away                        -0.017205   
over35_percentage                           -0.006765   
first_goal_home                              0.700183   

                             goals_conceded_min_0_to_10  \
goals_scored_min_0_to_10                      -0.271039   
goals_conceded_min_0_to_10                     1.000000   
goals_scored_min_11_to_20                     -0.022938   
goals_conceded_min_11_to_20                    0.032919   
wins_home                                      0.079044   
wins_away         

Métricas do modelo

In [17]:
# Avaliando o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibindo os resultados
print(f'Acurácia: {accuracy}')
print('Matriz de Confusão:')
print(conf_matrix)
print('Relatório de Classificação:')
print(class_report)

Acurácia: 0.8333333333333334
Matriz de Confusão:
[[2 1]
 [0 3]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [18]:
# Ajustar o parâmetro de profundidade máxima para evitar overfitting
model = RandomForestClassifier(max_depth=5)
model.fit(X_train, y_train)


RandomForestClassifier(max_depth=5)

Previsão de que time marcará o primeiro gol

In [19]:
def predict_first_goal(home_team_encoded, away_team_encoded, df_teams, model):
    # Filtrar as estatísticas do time da casa e visitante pelos IDs codificados
    home_stats = df_teams[df_teams['common_name_encoded'] == home_team_encoded].iloc[0]
    away_stats = df_teams[df_teams['common_name_encoded'] == away_team_encoded].iloc[0]

    # Criar um DataFrame para a nova partida
    match_data = pd.DataFrame({
        'goals_scored_min_0_to_10': [home_stats['goals_scored_min_0_to_10']],
        'goals_conceded_min_0_to_10': [away_stats['goals_conceded_min_0_to_10']],
        'goals_scored_min_11_to_20': [home_stats['goals_scored_min_11_to_20']],
        'goals_conceded_min_11_to_20': [away_stats['goals_conceded_min_11_to_20']],
        'wins_home': [home_stats['wins_home']],
        'wins_away': [away_stats['wins_away']],
        'loss_percentage_home': [home_stats['loss_percentage_home']],
        'loss_percentage_away': [away_stats['loss_percentage_away']],
        'over35_percentage': [home_stats['over35_percentage']]
    })

    # Fazer a previsão
    prediction = model.predict(match_data)
    # Retornar o resultado
    if prediction == 1:
        return f"O time da casa ({home_team_encoded}) marcará o primeiro gol."
    else:
        return f"O time visitante ({away_team_encoded}) marcará o primeiro gol."


Resultado do time que marcará o primeiro gol

In [20]:
home_team_encoded = 1 # Código do time da casa
away_team_encoded = 10  # Código do time visitante

result = predict_first_goal(home_team_encoded, away_team_encoded, df_teams, model)
print(result)


O time visitante (10) marcará o primeiro gol.
